In [1]:
import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggMapNet

np.random.seed(888) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

2022-10-07 13:25:05.018963: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
dfx = pd.read_excel('../CRC.xlsx', sheet_name='data')
dfx = dfx[dfx.columns[1:]]
dfx = np.log(dfx + 1e-8)

feature_groups_df = pd.read_excel('../CRC.xlsx', sheet_name='feature_info')
feature_group_list = feature_groups_df.Group.tolist()

dfy_all = pd.read_excel('../CRC.xlsx', sheet_name='sample_info')
dfy = pd.get_dummies(dfy_all['Group'])

X = dfx.values.astype(float)
Y = dfy.values.astype(float)
mp = AggMap(dfx, metric = 'correlation')

2022-10-07 13:25:08,143 - INFO - [bidd-aggmap] - Calculating distance ...
2022-10-07 13:25:08,160 - INFO - [bidd-aggmap] - the number of process is 16


100%|####################################################################################################################################| 849/849 [00:00<00:00, 5806.55it/s]


In [3]:
for me in ['random','mds', 'lle', 'umap', 'isomap', 'tsne']:
    
    mp.fit(cluster_channels = 5, emb_method=me, verbose = 0)
    country_idx = dfy_all.groupby(['Country']).apply(lambda x:x.index.tolist())
    country_names = country_idx.index.tolist()
    #study to study transfer

    run_all = []
    for train_country in country_names:

        train_idx = country_idx.loc[train_country]
        trainx = X[train_idx]
        trainY = Y[train_idx]
        trainX = mp.batch_transform(trainx, scale_method = 'standard')        
        clf = AggMapNet.MultiClassEstimator(epochs = 30,  batch_size = 1, 
                                           conv1_kernel_size = 5, 
                                           verbose = 0, gpuid = 1)
        clf.fit(trainX, trainY)  
        for test_country in country_names:
            test_idx = country_idx.loc[test_country]
            testY = Y[test_idx]
            testx = X[test_idx]
            testX = mp.batch_transform(testx, scale_method = 'standard')
            print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
            y_true = testY[:,0]
            y_pred = clf.predict(testX)[:,0]
            y_score = clf.predict_proba(testX)[:,0]
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
            acc = (tp + tn) / sum([tn, fp, fn, tp])
            sensitivity = tp / sum([tp, fn])
            specificity = tn / sum([tn, fp])
            prc_auc = prc_auc_score(y_true, y_score)
            roc_auc = roc_auc_score(y_true, y_score)
            precision = tp / sum([tp, fp])
            recall =  tp / sum([tp, fn]) #equals to sensitivity
            res = {'train_country': train_country,
                   'test_country':test_country,
                   'accuracy':acc, 
                   'prc_auc':prc_auc, 
                   'roc_auc':roc_auc,
                   'sensitivity': sensitivity, 
                   'specificity': specificity,
                   'precision':precision,
                   'recall':recall,
                  }
            run_all.append(res)

    df = pd.DataFrame(run_all)
    dfres = pd.crosstab(index = df.train_country, columns= df.test_country, 
                        values= df.roc_auc, aggfunc = np.mean)
    dfres = dfres.rename(columns = {'AUS':"AUS", 'CHI':'CHN', 'FRA':'FRA', 'GER':'DEU', 'USA':'USA'},
                        index =  {'AUS':"AUS", 'CHI':'CHN', 'FRA':'FRA', 'GER':'DEU', 'USA':'USA'}
                        )

    dfres.to_excel('./%s_study2study_results_ROC.xlsx' % me)

2022-10-07 13:25:37,998 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2022-10-07 13:25:38,016 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2022-10-07 13:25:39,020 - INFO - [bidd-aggmap] - Finished


100%|######################################################################################################################################| 109/109 [00:01<00:00, 62.47it/s]
2022-10-07 13:25:40.871017: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 13:25:40.888662: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 13:25:40.888753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


MultiClassEstimator(batch_norm=False, batch_size=1, conv1_kernel_size=5,
                    dense_avf='relu', dense_layers=[128], dropout=0.0,
                    epochs=30, gpuid='1', last_avf='softmax',
                    loss='categorical_crossentropy', lr=0.0001, metric='ACC',
                    monitor='val_loss', n_inception=2,
                    name='AggMap MultiClass Estimator', patience=10000,
                    random_state=32, verbose=0)


2022-10-07 13:25:40.889082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 13:25:40.890749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 13:25:40.890912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 13:25:40.890974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA


 input train and test X shape is (109, 30, 29, 5), (109, 30, 29, 5) 


...


 input train and test X shape is (104, 30, 29, 5), (114, 30, 29, 5) 


100%|####################################################################################################################################| 120/120 [00:00<00:00, 2356.15it/s]



 input train and test X shape is (104, 30, 29, 5), (120, 30, 29, 5) 


100%|####################################################################################################################################| 104/104 [00:00<00:00, 2212.58it/s]



 input train and test X shape is (104, 30, 29, 5), (104, 30, 29, 5) 
